# Filtering of features

Using the `metainfo` of the pre-processed data to filter the features. 

In [1]:
## system functions from roux
from roux.lib.sys import read_ps
from roux.lib.io import read_table
from roux.lib.io import to_table
## data functions from roux
import roux.lib.dfs as rd # attributes

In [ ]:
## parameters
input_path=None#metadata['metainfo']['filtered']['paralogs']
output_path=None#metadata['features']['filtered']['paralogs']
feature_paths=None#f"{metadata['pre_processed']['features']['path']}/*.csv"

### Metainfo

In [ ]:
df00=read_table(input_path)
df00.head(1)

In [ ]:
assert all(df00.groupby('pairs')['label'].nunique()==4)
assert df00['label common'].nunique()==4
assert all(df00.groupby('pairs')['label common'].nunique()==4)

### Features

In [ ]:
df01=read_table(
    read_ps(
        feature_paths,
    ),
    )
df01.head(1)

In [ ]:
## for merging the features
df01=(df01
    .assign(**{'cell id per subset':range(len(df01))})
    .rename(columns={str(i):f"feature #{i+1:03}" for i in range(128)},errors='raise')
    .log()  
)
df01.head(1)

### Merge

In [ ]:
df1=(df00
    .loc[:,
         [
        'pairs',
        'gene symbol query','gene symbol partner',
        'label','construct label','label common',
        'replicate','cell id per pair','cell id per subset',
        ]]
    .log.merge(
        right=df01,
        how='inner',
        on='cell id per subset',
        validate="1:1",
        validate_equal_length=True,
        suffixes=['',' features'],
    )
)
df1.head(1)

In [ ]:
if not 'control' in input_path:
    assert all(df1['label']==df1['label features'])

### Save

In [ ]:
to_table(df1,
         # metadata['features']['filtered']['paralogs'],
         output_path,
        )